In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import os
from moviepy import ImageClip, concatenate_videoclips
import matplotlib.image as mpimg

## Improvements to make:

* add sound
* make equal length of labels
* assign color to individual countries (or anything in the y axis)
* 

In [2]:
df_population = pd.read_csv("./population.csv")
df_population.sample(4)

,Entity,Year,all years
13827,Puerto Rico,2013,3606729
384,Americas (UN),1964,456963549
18105,Uruguay,1999,3253972
3230,Chad,1998,7938791


In [3]:
years = list(df_population.Year.unique())

In [4]:
# all_regions = list(df_population.Entity.unique())
# all_regions

In [ ]:
non_countries = [
    "Africa (UN)",
    "Americas (UN)",
    "Asia (UN)",
    "Europe (UN)",
    "High-income countries",
    "Land-locked developing countries (LLDC)",
    "Latin America and the Caribbean (UN)",
    "Least developed countries",
    "Less developed regions",
    "Less developed regions, excluding China",
    "Less developed regions, excluding least developed countries",
    "Low-income countries",
    "Lower-middle-income countries",
    "More developed regions",
    "Northern America (UN)",
    "Oceania (UN)",
    "Small island developing states (SIDS)",
    "Upper-middle-income countries",
    "World",
]

In [6]:
historical_max = df_population["all years"].max()
historical_max

np.int64(8091734853)

In [7]:
def get_top_ten(df, year, non_countries):
    df_cp = df.copy()
    df_cp = df_cp[df_cp["Year"] == year]
    total_population = df_cp.loc[df_cp["Entity"] == "World", "all years"].iloc[0]
    df_cp = df_cp[~df_cp["Entity"].isin(non_countries)]
    df_cp = df_cp.sort_values(by="all years", ascending=False)
    df_cp = df_cp.rename(columns={"Entity": "Country", "all years": "Population"})
    df_cp = df_cp.drop(columns=["Year"])
    df_cp["Percent of Total"] = round(df_cp["Population"] * 100 / total_population, 2)
    return df_cp.head(10).reset_index(drop=True), total_population

In [ ]:
def plot_top_ten(df_year, year, historical_max):
    with plt.xkcd():
        plt.figure(figsize=(9, 16), dpi=300)
        sns.barplot(
            data=df_year,
            y="Country",
            x="Population",
            hue="Country",
            palette="Set3",
            dodge=False,
            legend=False,
        )

        def human_format(x, pos):
            if x >= 1e9:
                return f"{x/1e9:.1f}B"
            return f"{int(x/1e6)}M"

        plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(human_format))
        plt.xlim(0, historical_max)

        plt.xlabel("Population")
        plt.ylabel("")
        plt.title(f"Top 10 countries - {year}", fontsize=14, weight="bold")

        plt.tight_layout()

        filename = f"./results/world_population_{year}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        plt.close()

In [9]:
# year = 2010
# df_year, world_population = get_top_ten(df_population, year, non_countries)
# plot_top_ten(df_year, year, 1500000000)

In [10]:
# for year in years:
#     df_year, world_population = get_top_ten(df_population, year, non_countries)
#     plot_top_ten(df_year, year, 1500000000)

In [ ]:
def images_to_video(
    folder="./results", output="./videos/world_population.mp4", duration=1
):
    """
    Turns all PNG images in a folder into a video using MoviePy.

    Args:
        folder (str): Folder with PNG images.
        output (str): Output video filename.
        duration (int or float): Seconds per image.
    """
    files = sorted([f for f in os.listdir(folder) if f.endswith(".png")])

    if not files:
        raise ValueError(f"No PNG files found in {folder}")
    clips = [ImageClip(os.path.join(folder, f), duration=duration) for f in files]
    video = concatenate_videoclips(clips, method="compose")
    video.write_videofile(output, fps=1)

In [12]:
images_to_video()

MoviePy - Building video ./videos/world_population.mp4.
MoviePy - Writing video ./videos/world_population.mp4



MoviePy - Done !
MoviePy - video ready ./videos/world_population.mp4
